<a href="https://colab.research.google.com/github/durfred/my-first-binder/blob/main/RAG_Alice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Update the path to your file in Drive
file_path = "/content/drive/MyDrive/my_stories/cleaned_alice_in_wonderland.txt"

with open(file_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()
print("Loaded text length:", len(raw_text))


Mounted at /content/drive
Loaded text length: 159206


In [2]:
!pip install -q langchain faiss-cpu gradio sentence-transformers llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct-GGUF/resolve/main/qwen2.5-1.5b-instruct-q6_k.gguf

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Split the text into manageable chunks
splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunks = splitter.split_text(raw_text)
documents = [Document(page_content=chunk) for chunk in chunks]

# Embed and build FAISS index
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_db = FAISS.from_documents(documents, embedding_model)

print(f"Total documents in FAISS: {len(documents)}")


In [7]:
from llama_cpp import Llama

llm = Llama(
    model_path="qwen2.5-1.5b-instruct-q6_k.gguf",  # path to GGUF file
    n_gpu_layers=0,   # set to >0 if using Colab GPU with CUDA build
    n_ctx=512,
    verbose=False
)


llama_context: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [8]:
def chat_rag(message, history):
    # Retrieve top-3 similar chunks
    retrieved_docs_with_scores = faiss_db.similarity_search_with_score(message, k=3)
    context = "\n\n".join([doc.page_content for doc, _ in retrieved_docs_with_scores])

    prompt = (
        "<|im_start|>system\n"
        "You are a helpful assistant who answers questions only based on the given context.\n"
        "If you don't know, say so clearly.<|im_end|>\n"
        f"<|im_start|>user\nContext:\n{context}\n\nQuestion: {message}<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    output = llm.create_completion(
        prompt=prompt,
        max_tokens=256,
        temperature=0.7,
        stop=["<|im_end|>"],
        echo=False
    )
    return output['choices'][0]['text'].strip()


In [9]:
import gradio as gr

iface = gr.ChatInterface(
    fn=chat_rag,
    title="📚 Alice in Wonderland RAG Chat (Qwen + FAISS)",
    description="Ask questions about Alice in Wonderland (cleaned). Qwen 1.5 answers using retrieved document context.",
    examples=[
        ["What happens when Alice falls down the rabbit hole?"],
        ["What is the Queen of Hearts like?"],
        ["Summarise the conversation with the Cheshire Cat."]
    ]
)

iface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d95288aa41467e09a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
